In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [3]:
## Read the ppdfs from the folder
loader=PyPDFDirectoryLoader("./pokemon guide")

documents=loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_documents=text_splitter.split_documents(documents)
final_documents[0:5]

[Document(page_content='Pokemo\x00\nScarle\x00\nAn\x00\nViole\x00\nWalkthroug\x00\nScarlet\nplayers\nwill\nhave\nthe\nopportunity\nto\ncatch\nand\nride\nK o\nraidon\n.\nViolet\nplayers\nwill\nbe\nable\nto\ncatch\nand\nride\nM i\nraidon\n.\nFirst\nthings\nﬁrst,\npick\nyour\nstarter!', metadata={'source': 'pokemon guide\\Pokemon Scarlet_Violet Walkthrough.pdf', 'page': 0}),
 Document(page_content='This\npokemon\ngame\nis\nset\nin\nthe\nregion\nof\nPaldea,\nafter\na\ngood\namount\nof\ntalking\nthrough\nthe\nbeginning\nof\nthe\ngame\nyou\nwill\nfind\nthere\nare\nthree\nmain\nstory-lines\nor\nQuests\nto\naccomplish,\nand\nthey\ncan\nbe\ndone\nin\nany\norder\nat\nany\ntime.\nThe\ngame\nallows\nyou\nto\njust\ngo\nexplore\nthe\nworld\nat\nyour\nown\npace.\nThis\ncan\nbe\na\nlittle\nconfusing\nthough\nsince\nit\ndoesn’t\ngive\nyou\na\nlot\nof\ndirection\nabout\nwhere\nto\ngo\nnext\nor\nwho\nto\nfight\nin\nwhat\norder,\nso\nI’m\ngoing\nto\nbreak\nit\ndown\nby\neach\nQuest\nline\nby\nthe\norder\n

In [4]:
len(final_documents)

375

In [6]:
# pip install sentence_transformers


     ---------------------------------------- 0.0/138.0 kB ? eta -:--:--
     -------------------------------------- 138.0/138.0 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 60.6/60.6 kB 3.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/171.5 kB ? eta -:--:--
   -------------------------------------- - 163.8/171.5 kB ? eta -:--:--
   ---------------------------------------- 171.5/171.5 kB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   -------- ------------------------------- 2.0/9.0 MB 41.8 MB/s eta 0:00:01
   --------------------- ------------------ 4.8/9.0 MB 51.2 MB/s eta 0:00:01
   ----------------------------------- ---- 8.0/9.0 MB 56.8 MB/s eta 0:00:01
   ---------------------------------------  9.0/9.0 MB 57.6 MB/s eta 0:00:01
   ---------------------------------------- 9.0/9.0 MB 41.0 MB/s eta 0:00:00
   -------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
catboost 1.2.5 requires matplotlib, which is not installed.
catboost 1.2.5 requires plotly, which is not installed.
pmdarima 2.0.4 requires statsmodels>=0.13.2, which is not installed.
shap 0.45.0 requires cloudpickle, which is not installed.
shap 0.45.0 requires numba, which is not installed.


In [7]:
## Embedding Using Huggingface
huggingface_embeddings=HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",      #sentence-transformers/all-MiniLM-l6-v2
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}

)

c:\Users\York Yong\OneDrive - Singapore Management University\Desktop\Langchain Project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\York Yong\OneDrive - Singapore Management University\Desktop\Langchain Project\venv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\York Yong\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks

In [8]:
import  numpy as np
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-6.07640184e-02 -3.67714763e-02 -3.10070696e-04 -5.13824038e-02
  3.01127620e-02  7.24598989e-02  3.06137372e-02  2.64649261e-02
 -6.03896342e-02 -1.74936056e-02  4.76792119e-02 -7.14672450e-03
  1.18155889e-02 -1.77500099e-02  1.85525585e-02  6.17035059e-03
  1.63524654e-02 -1.79625228e-02 -7.64947161e-02  7.11637810e-02
  3.24039757e-02  1.07135940e-02  2.78558694e-02 -6.16360679e-02
  2.21153013e-02  5.20732403e-02 -1.98273044e-02 -2.74009421e-03
 -3.45402621e-02 -1.61667719e-01 -1.72994956e-02 -1.63888093e-02
 -5.93208782e-02  8.04778561e-03 -3.06208320e-02  2.42936183e-02
  2.47084536e-02  2.45093275e-02 -1.52646110e-03  1.73146483e-02
  1.87328439e-02 -2.95198038e-02 -1.28317960e-02 -4.35282104e-03
 -1.56672373e-02  2.97725219e-02 -5.21790469e-03 -2.35111511e-04
  2.17873789e-02 -4.86199036e-02 -2.10014731e-02 -1.96895432e-02
 -3.14601511e-02  1.65789276e-02 -1.61911659e-02  1.51883177e-02
  8.89977887e-02  7.75640737e-03 -3.45064923e-02  5.28383404e-02
  6.14555739e-02  2.22838

In [9]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [10]:
## Query using Similarity Search
query="What are the starter pokemon?"
relevant_docments=vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

things
you
can
collect
in
this
first
zone!
Possible
team
pokemon:
Fidough,
The
new
Paldea
Wooper,
Houndour,
Fletchling,
Ralts,
Ghastly
will
appear
at
night.
All
these
are
pretty
good
to
get
early
and
fire
pokemon
can
be
tough
to
get
early.
There
is
also
Charcadet,
the
new
fire
pokemon
of
this
game
you
can
get
right
after
you
finish
talking
to
everyone
in
the
school.
So
try
to
think
about
how
you
might
want
to
build
your
team,
since
you
will
probably
want
someone
with
fire,
water,
electric
etc.
Especially
since
the
first
two
gyms
are
weak
to
fire.


In [11]:
retriever=vectorstore.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E80B91EA70> search_kwargs={'k': 3}


In [34]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']='<insert token here>'

In [35]:
from langchain_community.llms import HuggingFaceHub

hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}

)
query="What is the best starter pokemon?"
hf.invoke(query)

HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/mistralai/Mistral-7B-v0.1 (Request ID: F7lhN48nanc266d8zaJqG)

Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate

In [36]:
# #Hugging Face models can be run locally through the HuggingFacePipeline class.
# from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# hf = HuggingFacePipeline.from_model_id(
#     model_id="mistralai/Mistral-7B-v0.1",
#     task="text-generation",
#     pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
# )

# llm = hf 
# llm.invoke(query)

In [37]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [38]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [39]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [40]:

query="""How to defeat the first gym leader?"""

In [41]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/mistralai/Mistral-7B-v0.1 (Request ID: sqgWVX3-hw8-_j2cB7gzY)

Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate